<a href="https://colab.research.google.com/github/marcelaqs/Projeto-IV-2025/blob/main/projetoiv_dengue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# === DENGUE 2022–2024
# Montar Drive e defir a pasta do projeto.

from pathlib import Path
from google.colab import drive

drive.mount('/content/drive')

# caminho para a sua pasta no Drive:
PROJECT_DIR = Path('/content/drive/MyDrive/ProjetoIV_dengue')

DATA_DIR = PROJECT_DIR / 'data'
RAW_DIR  = DATA_DIR / 'raw'          # arquivos brutos
PROC_DIR = DATA_DIR / 'processed'    # saídas tratadas
LOG_DIR  = PROJECT_DIR / 'logs'      # manifestos e auditoria
OUT_DIR  = PROJECT_DIR / 'out'       # figuras/relatórios leves

for p in [RAW_DIR, PROC_DIR, LOG_DIR, OUT_DIR]:
    p.mkdir(parents=True, exist_ok=True)

print("Estrutura do projeto:")
for p in [PROJECT_DIR, RAW_DIR, PROC_DIR, LOG_DIR, OUT_DIR]:
    print(" -", p)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Estrutura do projeto:
 - /content/drive/MyDrive/ProjetoIV_dengue
 - /content/drive/MyDrive/ProjetoIV_dengue/data/raw
 - /content/drive/MyDrive/ProjetoIV_dengue/data/processed
 - /content/drive/MyDrive/ProjetoIV_dengue/logs
 - /content/drive/MyDrive/ProjetoIV_dengue/out


In [ ]:
# === CONFIGURAÇÃO DO PROJETO ===

import getpass, platform, json
from datetime import datetime

YEARS = [2022, 2023, 2024]


 # Nomes que eu vou usar para salvar/ler os brutos no Drive
 # URLs fixas (Open DataSUS / S3)

RAW_FILES =  {
  2022: "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR22.csv.zip",
  2023: "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR23.csv.zip",
  2024: "https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR24.csv.zip",
 }

In [ ]:
# === CONCATENAÇÃO EM STREAMING COM ESQUEMA FIXO (ROBUSTO ENTRE ANOS) ===
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path

CSV_BY_YEAR = {
    2022: RAW_DIR / "DENGBR22.csv",
    2023: RAW_DIR / "DENGBR23.csv",
    2024: RAW_DIR / "DENGBR24.csv",
}

PARQUET_UNICO = PROC_DIR / "dengue_2022_2024_bruto.parquet"
CHUNK = 200_000  # pode reduzir para 100_000 se quiser usar menos RAM

# 1) Descubro o conjunto-unificado de colunas (apenas cabeçalho, sem carregar dados)
cols_union = []
seen = set()
for yr, path in CSV_BY_YEAR.items():
    if not path.exists():
        raise FileNotFoundError(f"Arquivo não encontrado: {path}")
    hdr = pd.read_csv(path, sep=",", encoding="utf-8", nrows=0)
    for c in hdr.columns.tolist():
        if c not in seen:
            seen.add(c)
            cols_union.append(c)

# acrescento 'ano_origem' ao final (não existe no bruto)
if "ano_origem" not in cols_union:
    cols_union.append("ano_origem")

# 2) Defino um schema fixo: todas as colunas como string, exceto 'ano_origem' (int64)
fields = []
for c in cols_union:
    if c == "ano_origem":
        fields.append(pa.field(c, pa.int64()))
    else:
        fields.append(pa.field(c, pa.string()))
schema = pa.schema(fields)

# Se já existe, apago para reescrever do zero
if PARQUET_UNICO.exists():
    PARQUET_UNICO.unlink()

writer = None
linhas_por_ano = {yr: 0 for yr in CSV_BY_YEAR}

def _coerce_chunk_to_schema(df_chunk: pd.DataFrame) -> pa.Table:
    """
    - Garante TODAS as colunas (reindex no superset).
    - Converte todas as colunas (exceto ano_origem) para pandas StringDtype().
    - ano_origem -> Int64 (pandas) para ir como int64 no Arrow.
    - Converte para Arrow Table usando o schema fixo.
    """
    # garante superset (colunas que não existem viram NA)
    df_chunk = df_chunk.reindex(columns=cols_union, fill_value=pd.NA)

    # tipos pandas estáveis
    for c in cols_union:
        if c == "ano_origem":
            # garante numérico inteiro com NA seguro
            df_chunk[c] = pd.to_numeric(df_chunk[c], errors="coerce").astype("Int64")
        else:
            # tudo como string (mesmo que a coluna esteja toda vazia)
            if df_chunk[c].dtype.name != "string":
                df_chunk[c] = df_chunk[c].astype("string")

    # converto para Arrow com o schema alvo (evita colunas 'null')
    table = pa.Table.from_pandas(df_chunk, schema=schema, preserve_index=False)
    return table

# 3) Escrevo em streaming com schema fixo
for yr, csv_path in CSV_BY_YEAR.items():
    print(f"→ Processando {csv_path.name} em chunks de {CHUNK:,} linhas...")
    for i, chunk in enumerate(pd.read_csv(csv_path, sep=",", encoding="utf-8",
                                          dtype=str, low_memory=False, chunksize=CHUNK)):
        # adiciono a coluna do ano de origem
        chunk["ano_origem"] = yr
        linhas_por_ano[yr] += len(chunk)

        table = _coerce_chunk_to_schema(chunk)

        if writer is None:
            writer = pq.ParquetWriter(PARQUET_UNICO, schema, compression="snappy")

        writer.write_table(table)

        if (i + 1) % 5 == 0:
            print(f"   · {csv_path.name}: {((i+1)*CHUNK):,} linhas já escritas...")

if writer is not None:
    writer.close()

print("✓ Parquet único salvo em:", PARQUET_UNICO)
print("Linhas por ano:", {k: f'{v:,}' for k, v in linhas_por_ano.items()})
print("Total de linhas:", f"{sum(linhas_por_ano.values()):,}")



→ Processando DENGBR22.csv em chunks de 200,000 linhas...
   · DENGBR22.csv: 1,000,000 linhas já escritas...
→ Processando DENGBR23.csv em chunks de 200,000 linhas...
   · DENGBR23.csv: 1,000,000 linhas já escritas...
→ Processando DENGBR24.csv em chunks de 200,000 linhas...
   · DENGBR24.csv: 1,000,000 linhas já escritas...
   · DENGBR24.csv: 2,000,000 linhas já escritas...
   · DENGBR24.csv: 3,000,000 linhas já escritas...
   · DENGBR24.csv: 4,000,000 linhas já escritas...
   · DENGBR24.csv: 5,000,000 linhas já escritas...
   · DENGBR24.csv: 6,000,000 linhas já escritas...
✓ Parquet único salvo em: /content/drive/MyDrive/ProjetoIV_dengue/data/processed/dengue_2022_2024_bruto.parquet
Linhas por ano: {2022: '1,393,877', 2023: '1,508,653', 2024: '6,434,137'}
Total de linhas: 9,336,667


In [ ]:
# === INSPEÇÃO LEVE DO PARQUET (HEAD + CONTAGEM POR ano_origem) ===
import pyarrow.parquet as pq


PARQUET_UNICO = PROC_DIR / "dengue_2022_2024_bruto.parquet"
pf = pq.ParquetFile(PARQUET_UNICO)

# 1) HEAD seguro (~5 linhas) lendo o primeiro(s) row group(s)
n_quero = 5
coletados, lidos = [], 0
for rg in range(pf.num_row_groups):
    df_rg = pf.read_row_group(rg).to_pandas()
    coletados.append(df_rg)
    lidos += len(df_rg)
    if lidos >= n_quero:
        break
df_head = pd.concat(coletados, ignore_index=True).head(n_quero)
print("Head do parquet concatenado:")
display(df_head)

# 2) Contagem por ano_origem (lendo só a coluna, super leve)
anos = pf.read(columns=["ano_origem"]).to_pandas()
contagem_ano_origem = anos["ano_origem"].value_counts().sort_index()
print("\nNotificações por ano_origem (do parquet):")
display(contagem_ano_origem)


Head do parquet concatenado:


,TP_NOT,ID_AGRAVO,DT_NOTIFIC,SEM_NOT,NU_ANO,SG_UF_NOT,ID_MUNICIP,ID_REGIONA,ID_UNIDADE,DT_SIN_PRI,...,PLAQ_MENOR,CON_FHD,COMPLICA,TP_SISTEMA,NDUPLIC_N,DT_DIGITA,CS_FLXRET,FLXRECEBI,MIGRADO_W,ano_origem
0,2,A90,2022-01-12,202202,2022,12,120020,1941,6801099,2022-01-06,...,None,None,None,2,None,2022-01-17,None,None,None,2022
1,2,A90,2022-08-11,202232,2022,12,120020,1941,5336171,2022-08-03,...,None,None,None,2,None,2022-08-12,None,None,None,2022
2,2,A90,2022-03-25,202212,2022,12,120020,1941,2002116,2022-03-21,...,None,None,None,2,None,2022-03-31,None,None,None,2022
3,2,A90,2022-03-07,202210,2022,12,120020,1941,6801099,2022-03-04,...,None,None,None,2,None,2022-03-23,None,None,None,2022
4,2,A90,2022-01-18,202203,2022,12,120020,1941,6801099,2022-01-18,...,None,None,None,2,None,2022-01-26,None,None,None,2022



Notificações por ano_origem (do parquet):


,count
ano_origem,
2022,1393877
2023,1508653
2024,6434137
